# Imports

In [1]:
import sys

modules_to_reload = [
    "src.utils.method_loggers",
    "src.utils.method_runners",
    "src.utils.metrics_calculators",
    "src.utils.tensor_handlers",
    "src.utils.trackers",
    "src.utils.video_controller",
]

for module in modules_to_reload:
    if module in sys.modules:
        del sys.modules[module]

%load_ext memory_profiler
%load_ext autoreload
%autoreload 2

import contextlib
import gc
import os
from itertools import product

import numpy as np
import tensorflow as tf
from numba import cuda

tf.random.set_seed(42)

import t3f
import tensorly as tl
import torch
from dotenv import load_dotenv
from tqdm import tqdm

from src.utils.image_controller import download_image, extract_image_frames
from src.utils.method_loggers import MethodLogger
from src.utils.method_runners import MethodRunner
from src.utils.read_logs import LogReader
from src.utils.save_frames import SaveFramesFactory
from src.utils.tensor_handlers import normalize_frames
from src.utils.trackers import GPUTensorflowMemoryTracker, GPUTorchMemoryTracker, RAMMemoryTracker, TimeTracker
from src.utils.video_controller import download_youtube_video, extract_frames

load_dotenv()

2024-10-17 20:55:15.988371: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-17 20:55:16.140139: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-17 20:55:16.179049: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-17 20:55:16.446968: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-17 20:55:18.124572: W tensorflow/compiler/tf2

True

In [2]:
torch_device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch_device

device(type='cuda')

In [3]:
torch.cuda.empty_cache()

In [4]:
tf_physical_device = tf.config.list_physical_devices("GPU")[0].name
tf_device = ":".join(tf_physical_device.split(":")[1:3])
tf_devices = [tf_device]
tf_physical_device

I0000 00:00:1729180524.801251     884 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1729180525.884272     884 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1729180525.884355     884 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


'/physical_device:GPU:0'

# Get tensors

## Some params

In [5]:
video_url = "https://www.youtube.com/watch?v=eSKe2Vx-rpY"
proxy_url = os.getenv("PROXY_URL")
cache_dir_video = "../.cache/video"

image_url = "https://i.pinimg.com/564x/04/b2/68/04b26838bdd5e2ba54d0144558685bae.jpg"
cache_dir_image = "../.cache/image"

log_file_path = "../.cache/method_logs.json"

method_logs_list = []

## Get some tensors from different data types

### Video

In [6]:
video_path = download_youtube_video(video_url, cache_dir=cache_dir_video.__str__(), proxy_url=proxy_url)

Видео уже загружено и закешировано: ../.cache/video/eSKe2Vx-rpY.mp4


In [7]:
video_frames, original_fps, frame_size = extract_frames(video_path)

In [8]:
video_frames.shape

(440, 640, 360, 3)

### Image

In [9]:
image_path = download_image(image_url, cache_dir_image)
image_frames = extract_image_frames(image_path)

Изображение уже загружено и закешировано: ../.cache/image/04b26838bdd5e2ba54d0144558685bae.jpg


In [10]:
image_frames.shape

(564, 564, 3)

# Theoretical comparison of some libraries and implementations

Some packages which can decompose some dense type of tensors, from [this](https://arxiv.org/pdf/2103.13756) paper


Decomposition methods which used:
1. Canonical Polyadic Decomposition as PARAllel FACtors analysis (aka PARAFAC aka CPD aka CP)
2. Tucker Decomposition
3. Tensor Train
4. some variants of its (Other)

Tensor types:
1. Dense (D)
2. Sparse (S)
3. BlockSparse (BS)
4. Symmetric
5. Supersymmetric

Target system:
1. CPU (C)
2. GPU(G)
3. Distributed Memory (D)



| Method name                                                                             | Decomposition methods implemented | Tensor Type | Platform | Language         | Git | PyPI | Want to check | Checked     |
|-----------------------------------------------------------------------------------------|-----------------------------------|------------|----------|------------------|------|------|--------------|-------------|
| [AdaTM](https://github.com/hpcgarage/AdaTM)                                             | CP                                | S          | C        | C                | +    | ?    |              |             |
| [BTAS](https://github.com/ValeevGroup/BTAS)                                             | CP, Tucker                        | nan        | C        | C++              | +    | ?    |              |             |
| [CP-CALS](https://github.com/HPAC/CP-CALS)                                              | CP, Other                         | D          | C, G     | C++, Mat         | +    |      | +            |             |
| [CSTF](https://github.com/ZacBlanco/cstf)                                               | Other                             | S          | D        | Scala            | +    | ?    |              |             |
| [D-Tucker](https://datalab.snu.ac.kr/dtucker/resources/DTucker-v1.0.tar.gz)             | Tucker, Other                     | D          | C        | Matlab           |      | ?    |              |             |
| [DFacTo](http://www.joonheechoi.com/research.)                                          | CP                                | S          | C, D     | C++              |      | ?    |              |             |
| [EXATN](https://github.com/ORNL-QCI/exatn)                                              | TensorTrain                       | D          | C, D, G  | C++, Py          | +    |      | +            |             |
| [Genten](https://gitlab.com/tensors/genten)                                             | CP                                | D, S       | C, G     | C++              | +    |      | +            |             |
| GigaTensor                                                                              | CP                                | D          | C        | C++, Python      |      | ?    |              |             |
| [ITensor](https://github.com/ITensor/ITensor)                                           | TensorTrain                       | D, BS      | C, G     | C++, Julia       | +    |      | +            |             |
| [multiway](https://cran.r-project.org/web/packages/multiway/index.html)                 | CP, Tucker, Other                 | D          | C        | R                |      | ?    |              |             |
| [N-way toolbox](http://www.models.life.ku.dk/nwaytoolbox/download)                      | CP, Tucker, Other                 | D          | C        | Matlab           |      | ?    |              |             |
| [ParCube](https://www.cs.ucr.edu/~epapalex/src/parCube.zip)                             | CP                                | S          | C        | Matlab           |      | ?    |              |             |
| [ParTensor](https://github.com/neurocom/partensor-toolbox)                              | CP                                | D          | C, G     | C++              | +    |      | +            |             |
| [ParTI!](https://github.com/hpcgarage/ParTI)                                            | CP, Tucker                        | S          | C, G     | C, CUDA, Mat     | +    | ?    |              |             |
| [PLANC](https://github.com/ramkikannan/planc)                                           | CP                                | S          | C, D     | C++              | +    | ?    |              |             |
| [PLS toolbox](https://eigenvector.com/software/pls-toolbox/)                            | CP          , Tucker              | D          | C        | Matlab           |      | ?    |              |             |
| [Pytensor](https://code.google.com/archive/p/pytensor/source/default/source)            | Tucker                            | D, S       | C        | Python           |      | ?    |              |             |
| [rTensor](https://github.com/jamesyili/rTensor)                                         | CP, Tucker, Other                 | D          | C        | R                | +    |      | +            |             |
| [rTensor (randomized)](https://github.com/erichson/rTensor)                             | CP                                | D          | C        | Python           | +    |      | +       +    |             |
| [scikit-tensor](https://github.com/mnick/scikit-tensor)                                 | CP, Tucker, Other                 | D, S       | C        | Python           | +    | +    | +    +   +   | too old     |
| [Scikit-TT](https://github.com/PGelss/scikit_tt)                                        | TensorTrain                       | D          | C        | Python           | +    |      |     +   +    |             |
| [SPALS](https://github.com/dehuacheng/SpAls)                                            | CP                                | S          | C        | C++              | +    | ?    |              |             |
| [SPARTan](https://github.com/kperros/SPARTan)                                           | Other                             | S          | C        | Matlab           | +    | ?    |              |             |
| [SPLATT](https://github.com/ShadenSmith/splatt)                                         | CP                                | S          | C, D     | C, C++, Oct, Mat | +    | ?    |              |             |
| [SuSMoST](https://susmost.com/downloads.html)                                           | TensorTrain, Other                | D          | C        | Python           |      | ?    |              |             |
| [T3F](https://github.com/Bihaqo/t3f)                                                    | TensorTrain                       | D          | C, G     | Python           | +    | +    | +    + +     | in progress |
| [TDALAB](https://github.com/andrewssobral/TDALAB)                                       | CP                                | D, S       | C        | Python, Matlab   | +    |      | +         +  |             |
| [TeNPy](https://github.com/tenpy/tenpy)                                        | TensorTrain                       | D          | C        | Python           | +    | +    | +      + +   | in progress |
| [Tensor Fox](https://github.com/felipebottega/Tensor-Fox)                               | CP                                | D, S       | C        | Python, Matlab   | +    | +    | +    + +     | ?           |
| [Tensor package](http://www.gipsa-lab.fr/~pierre.comon/TensorPackage/tensorPackage.html) | CP                                | D          | C        | Matlab           |      | ?    |              |             |
| [Tensor Toolbox](https://gitlab.com/tensors/tensor_toolbox)                             | CP, Tucker, Other                 | D, S       | C        | Matlab           | +    |      | +            |             |
| [tensor_decomposition](https://github.com/cyclops-community/tensor_decomposition)       | CP, Tucker                        | D          | C, D     | Python           | +    |      | +        +   |             |
| [TensorBox](https://github.com/phananhhuy/TensorBox)                                    | CP, Tucker, Other                 | D, S       | C        | Matlab           | +    |      | +            |             |
| [TensorD](https://github.com/Large-Scale-Tensor-Decomposition/tensorD)                  | CP, Tucker                        | D          | C, G     | Python           | ?    | ?    |              |             |
| [TensorLab](https://www.tensorlab.net)                                                  | CP, Tucker, Other                 | D, S       | C        | Matlab           |      | ?    |              |             |
| [TensorLab+](https://www.tensorlabplus.net)                                             | CP, Other                         | D, S       | C        | Matlab           |      | ?    |              |             |
| [TensorLy](https://github.com/tensorly/tensorly)                                        | CP, Tucker, TensorTrain, Other    | D          | C, G     | Python           | +    | +    | +       + +  | in progress |
| [Three-Way](https://github.com/cran/ThreeWay)                                           | CP, Tucker                        | D          | C        | R                | +    |      | +            |             |
| [TNR](https://github.com/ycyuustc/matlab)                                               | Other                             | D          | C        | Matlab           | +    |      | +            |             |
| [TT-Toolbox](https://github.com/oseledets/TT-Toolbox)                                   | TensorTrain                       | D          | C, D, G  | Matlab, Python   | +    |      | +       +    |             |
| [xerus](https://git.hemio.de/xerus/xerus/)                                              | TensorTrain                       | D, S       | C        | C++              | +    |      | +            |             |

# Implementations of Decompositions methods

## TensorLy

In [11]:
# {‘numpy’, ‘mxnet’, ‘pytorch’, ‘tensorflow’, ‘cupy’}
# backend variants for tensorly
# tl.set_backend('pytorch')
# with tl.backend_context(‘pytorch’): ... pass

# video_frames_cuda = tl.tensor(video_frames.copy()).to(device)
# video_frames_cuda = tl.tensor(video_frames.copy())

### Tucker (tl.decomposition.tucker)

In [12]:
logs = LogReader.load_logs_from_file(log_file_path)

#### Params

In [13]:
tl.SVD_FUNS

['truncated_svd', 'symeig_svd', 'randomized_svd']

In [14]:
tensor_rank_mapping = {
    "image": (image_frames.shape[0] // 2, image_frames.shape[1] // 2, image_frames.shape[2]),
    "video": (video_frames.shape[0] // 2, video_frames.shape[1], video_frames.shape[2], video_frames.shape[3]),
}

input_tensors = {
    "image": image_frames.copy(),
    "video": video_frames.copy(),
}

n_iter_max_param = 100

svd_params = ["truncated_svd", "symeig_svd", "randomized_svd"]

init_params = ["svd", "random"]

backend_params = ["pytorch", "numpy"]

random_state_param = 42

total_iterations = len(list(product(svd_params, init_params, backend_params))) * len(input_tensors)

In [15]:
# with tl.backend_context('pytorch'):
#     test = tl.tensor(image_frames.copy()).to(torch_device)
#     tl.decomposition.tucker(test, rank={image_frames.shape[0] // 2, image_frames.shape[1], image_frames.shape[2])

#### Compare method with some params and log it

In [16]:
for data_type_name, input_tensor in input_tensors.items():
    rank_param = tensor_rank_mapping[data_type_name]

    for backend, svd_func, init_method in tqdm(
        product(backend_params, svd_params, init_params), desc="Проверка набора параметров", total=total_iterations
    ):
        library_method_name = "TensorLy_Tucker"
        method_name = f"{library_method_name}_{data_type_name}_{backend}_{svd_func}_{init_method}"

        if logs:
            existing_log = next(
                (
                    log
                    for log in logs
                    if log["method_name"] == method_name
                    and log["method_args"].get("init") == init_method
                    and log["method_args"].get("svd") == svd_func
                    and log["qualitative_metrics"].get("TensorLy backend") == backend
                ),
                None,
            )
            if existing_log:
                print(f"Пропущена итерация: логи уже существуют для {method_name}")
                continue

        torch.cuda.synchronize()
        torch.cuda.empty_cache()
        gc.collect()

        try:
            with tl.backend_context(backend):
                if backend == "pytorch":
                    tensor_param = tl.tensor(input_tensor).to(torch_device)
                elif backend == "numpy" or backend is None:
                    tensor_param = tl.tensor(input_tensor)

                method_runner = MethodRunner(
                    func=tl.decomposition.tucker,
                    method_input_tensor=tensor_param,
                    library_method_name=library_method_name,
                    backend_name=backend,
                    gpu_memory_tracker=GPUTorchMemoryTracker(),
                    ram_memory_tracker=RAMMemoryTracker(),
                    time_tracker=TimeTracker(),
                )

                method_logger = MethodLogger(
                    method_name=method_name,
                    qualitative_metrics={
                        "Language": "Python",
                        "Library": "TensorLy",
                        "TensorLy backend": f"{backend}",
                        "Tensor type": "Dense",
                        "Data type": data_type_name,
                        "Platform": "CPU, GPU",
                        "Decomposition method": "Tucker",
                    },
                    method_args={
                        "tensor": tensor_param,
                        "rank": rank_param,
                        "n_iter_max": n_iter_max_param,
                        "init": init_method,
                        "svd": svd_func,
                        "random_state": random_state_param,
                    },
                    runner=method_runner,
                )

            reconstructed_tensor = method_runner.reconstructed_tensor

            method_logger.save_logs_to_file(is_test=False)

            reconstructed_frames = []

            reconstructed_tensor = reconstructed_tensor.cpu().numpy() if backend == "pytorch" else reconstructed_tensor
            reconstructed_frames = np.array([normalize_frames(frame) for frame in reconstructed_tensor])

            save_params_combinations = {
                "image": {"name": method_logger.name, "frames": reconstructed_frames},
                "video": {"name": method_logger.name, "frames": reconstructed_frames, "fps": original_fps, "frame_size": frame_size},
            }
            save_params = save_params_combinations[data_type_name]

            SaveFramesFactory.get_save_methods(frame_name=data_type_name).save_frames(**save_params)

        except (torch.cuda.OutOfMemoryError, MemoryError) as e:
            print(f"Пропущена итерация из-за недостатка памяти: {backend}, {svd_func}, {init_method}. Ошибка: {e!s}")
            torch.cuda.synchronize()
            torch.cuda.empty_cache()
            gc.collect()
            continue

Эксперимент набора параметров:   0%|          | 0/10 [00:00<?, ?it/s]/home/johndoe_19/git-projects/tensor-methods-comparison/.venv/lib/python3.11/site-packages/torch/distributed/distributed_c10d.py:293: UserWarning: torch.distributed.reduce_op is deprecated, please use torch.distributed.ReduceOp instead
  warnings.warn(

Проверка набора параметров:   4%|▍         | 1/24 [00:52<19:59, 52.16s/it]

Изображение сохранено как /home/johndoe_19/git-projects/tensor-methods-comparison/.cache/output/TensorLy_Tucker_image_pytorch_truncated_svd_svd.jpg



Проверка набора параметров:   8%|▊         | 2/24 [01:30<16:07, 43.99s/it]

Изображение сохранено как /home/johndoe_19/git-projects/tensor-methods-comparison/.cache/output/TensorLy_Tucker_image_pytorch_truncated_svd_random.jpg



Проверка набора параметров:  12%|█▎        | 3/24 [01:37<09:26, 27.00s/it]

...

Пропущена итерация: логи уже существуют для TensorLy_Tucker_video_pytorch_truncated_svd_svd
Пропущена итерация: логи уже существуют для TensorLy_Tucker_video_pytorch_truncated_svd_random



Проверка набора параметров:  12%|█▎        | 3/24 [00:02<00:14,  1.45it/s]

Пропущена итерация из-за недостатка памяти: pytorch, symeig_svd, svd. Ошибка: CUDA out of memory. Tried to allocate 1779.79 GiB (GPU 0; 11.00 GiB total capacity; 1.14 GiB already allocated; 8.75 GiB free; 1.15 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF
Пропущена итерация: логи уже существуют для TensorLy_Tucker_video_pytorch_symeig_svd_random
Пропущена итерация: логи уже существуют для TensorLy_Tucker_video_pytorch_randomized_svd_svd
Пропущена итерация: логи уже существуют для TensorLy_Tucker_video_pytorch_randomized_svd_random
Пропущена итерация: логи уже существуют для TensorLy_Tucker_video_numpy_truncated_svd_svd
Пропущена итерация: логи уже существуют для TensorLy_Tucker_video_numpy_truncated_svd_random



Проверка набора параметров:  50%|█████     | 12/24 [00:02<00:02,  4.10it/s]

Пропущена итерация из-за недостатка памяти: numpy, symeig_svd, svd. Ошибка: Unable to allocate 445. GiB for an array with shape (691200, 691200) and data type uint8
Пропущена итерация: логи уже существуют для TensorLy_Tucker_video_numpy_symeig_svd_random
Пропущена итерация: логи уже существуют для TensorLy_Tucker_video_numpy_randomized_svd_svd
Пропущена итерация: логи уже существуют для TensorLy_Tucker_video_numpy_randomized_svd_random


#### Clear cache and gc collect

In [17]:
torch.cuda.synchronize()
torch.cuda.empty_cache()

variables_to_delete = [
    "method_runner",
    "method_logger",
    "tensor_param",
    "logs",
    "tensor_rank_mapping",
    "input_tensors",
    "n_iter_max_param",
    "svd_params",
    "init_params",
    "backend_params",
    "random_state_param",
    "total_iterations",
    "data_type_name",
    "input_tensor",
    "rank_param",
    "backend",
    "svd_func",
    "init_method",
    "library_method_name",
    "method_name",
    "reconstructed_tensor",
    "reconstructed_frames",
    "save_params_combinations",
    "save_params",
    "frame",
]

for var in variables_to_delete:
    with contextlib.suppress(KeyError):
        del globals()[var]

gc.collect()

485

### Tensor Train - MPS (tensorly.decomposition.tensor_train)

In [18]:
logs = LogReader.load_logs_from_file(log_file_path)

#### Params

In [19]:
tensor_rank_mapping = {
    "image": [1, 212, 212, 1],
    "video": [1, 500, 302, 500, 1],
}

input_tensors = {
    "image": image_frames.copy(),
    "video": video_frames.copy(),
}

svd_params = ["truncated_svd", "symeig_svd", "randomized_svd"]

backend_params = ["pytorch", "numpy"]

total_iterations = len(list(product(backend_params, svd_params))) * len(input_tensors)

#### Compare method with some params and log it

In [20]:
for data_type_name, input_tensor in input_tensors.items():
    rank_param = tensor_rank_mapping[data_type_name]
    for backend, svd_func in tqdm(product(backend_params, svd_params), desc="Проверка набора параметров", total=total_iterations):
        library_method_name = "TensorLy_TensorTrain"
        method_name = f"{library_method_name}_{data_type_name}_{backend}_{svd_func}"

        if logs:
            existing_log = next(
                (
                    log
                    for log in logs
                    if log["method_name"] == method_name
                    and log["method_args"].get("svd") == svd_func
                    and log["qualitative_metrics"].get("TensorLy backend") == backend
                ),
                None,
            )
            if existing_log:
                print(f"Пропущена итерация: логи уже существуют для {method_name}")
                continue

        gc.collect()
        torch.cuda.empty_cache()
        torch.cuda.synchronize()

        try:
            with tl.backend_context(backend):
                if backend == "pytorch":
                    tensor_param = tl.tensor(input_tensor).to(torch_device)
                elif backend == "numpy" or backend is None:
                    tensor_param = tl.tensor(input_tensor)

                method_runner = MethodRunner(
                    func=tl.decomposition.tensor_train,
                    method_input_tensor=tensor_param,
                    library_method_name=library_method_name,
                    backend_name=backend,
                    gpu_memory_tracker=GPUTorchMemoryTracker(),
                    ram_memory_tracker=RAMMemoryTracker(),
                    time_tracker=TimeTracker(),
                )

                method_logger = MethodLogger(
                    method_name=method_name,
                    qualitative_metrics={
                        "Language": "Python",
                        "Library": "TensorLy",
                        "TensorLy backend": f"{backend}",
                        "Tensor type": "Dense",
                        "Data type": data_type_name,
                        "Platform": "CPU, GPU",
                        "Decomposition method": "TensorTrain",
                    },
                    method_args={
                        "input_tensor": tensor_param,
                        "rank": rank_param,
                        "svd": svd_func,
                    },
                    runner=method_runner,
                )

            reconstructed_tensor_from_tt_factors = method_runner.reconstructed_tensor

            method_logger.save_logs_to_file(is_test=False)

            reconstructed_frames = []

            for tt_factor in reconstructed_tensor_from_tt_factors:
                if backend == "pytorch":
                    reconstructed_frames.append(normalize_frames(tt_factor.cpu().numpy()))
                else:
                    reconstructed_frames.append(normalize_frames(tt_factor))
            reconstructed_frames = np.array(reconstructed_frames)

            save_params_combinations = {
                "image": {"name": method_logger.name, "frames": reconstructed_frames},
                "video": {"name": method_logger.name, "frames": reconstructed_frames, "fps": original_fps, "frame_size": frame_size},
            }
            save_params = save_params_combinations[data_type_name]

            SaveFramesFactory.get_save_methods(frame_name=data_type_name).save_frames(**save_params)

        except (torch.cuda.OutOfMemoryError, MemoryError) as e:
            print(f"Пропущена итерация из-за недостатка памяти: {backend}, {svd_func}. Ошибка: {e!s}")
            gc.collect()
            torch.cuda.empty_cache()
            torch.cuda.synchronize()
            continue

Проверка набора параметров:   8%|▊         | 1/12 [00:10<01:58, 10.80s/it]

Изображение сохранено как /home/johndoe_19/git-projects/tensor-methods-comparison/.cache/output/TensorLy_TensorTrain_image_pytorch_truncated_svd.jpg



Проверка набора параметров:  17%|█▋        | 2/12 [00:17<01:23,  8.33s/it]

Пропущена итерация из-за недостатка памяти: pytorch, symeig_svd. Ошибка: CUDA out of memory. Tried to allocate 53.26 GiB (GPU 0; 11.00 GiB total capacity; 24.52 MiB already allocated; 9.86 GiB free; 42.00 MiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF



Проверка набора параметров:  25%|██▌       | 3/12 [00:27<01:21,  9.09s/it]

...

Пропущена итерация: логи уже существуют для TensorLy_TensorTrain_video_pytorch_truncated_svd



Проверка набора параметров:  17%|█▋        | 2/12 [00:02<00:10,  1.00s/it]

Пропущена итерация из-за недостатка памяти: pytorch, symeig_svd. Ошибка: CUDA out of memory. Tried to allocate 1779.79 GiB (GPU 0; 11.00 GiB total capacity; 1.14 GiB already allocated; 8.75 GiB free; 1.15 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF
Пропущена итерация: логи уже существуют для TensorLy_TensorTrain_video_pytorch_randomized_svd
Пропущена итерация: логи уже существуют для TensorLy_TensorTrain_video_numpy_truncated_svd



Проверка набора параметров:  50%|█████     | 6/12 [00:02<00:02,  2.08it/s]

Пропущена итерация из-за недостатка памяти: numpy, symeig_svd. Ошибка: Unable to allocate 445. GiB for an array with shape (691200, 691200) and data type uint8
Пропущена итерация: логи уже существуют для TensorLy_TensorTrain_video_numpy_randomized_svd


#### Clear cache and gc collect

In [21]:
torch.cuda.synchronize()
torch.cuda.empty_cache()

variables_to_delete = [
    "logs",
    "tensor_rank_mapping",
    "input_tensors",
    "svd_params",
    "backend_params",
    "total_iterations",
    "data_type_name",
    "input_tensor",
    "rank_param",
    "backend",
    "svd_func",
    "library_method_name",
    "method_name",
    "reconstructed_tensor_from_tt_factors",
    "method_runner",
    "method_logger",
    "reconstructed_frames",
    "save_params_combinations",
    "save_params",
    "tt_factor",
]

for var in variables_to_delete:
    with contextlib.suppress(KeyError):
        del globals()[var]

gc.collect()

485

## T3F

### Tensor Train

In [22]:
logs = LogReader.load_logs_from_file(log_file_path)

#### Params

In [23]:
tensor_rank_mapping = {
    "image": [1, 212, 212, 1],
    "video": [1, 500, 302, 500, 1],
}

input_tensors = {
    "image": image_frames.copy().astype(np.float32),
    "video": video_frames.copy().astype(np.float32),
}

backend_param = "tensorflow"

total_iterations = len(input_tensors)

#### Compare method with some params and log it

In [24]:
for data_type_name, input_tensor in tqdm(input_tensors.items(), desc="Проверка набора параметров", total=total_iterations):
    rank_param = tensor_rank_mapping[data_type_name]
    tensor_param = input_tensor

    library_method_name = "T3F_TensorTrain"
    method_name = f"{library_method_name}_{data_type_name}"

    if logs:
        existing_log = next((log for log in logs if log["method_name"] == method_name), None)
        if existing_log:
            print(f"Пропущена итерация: логи уже существуют для {method_name}")
            continue

    gc.collect()

    method_runner = MethodRunner(
        func=t3f.to_tt_tensor,
        method_input_tensor=tensor_param,
        library_method_name=library_method_name,
        backend_name=backend_param,
        gpu_memory_tracker=GPUTensorflowMemoryTracker(tf_devices=tf_devices),
        ram_memory_tracker=RAMMemoryTracker(),
        time_tracker=TimeTracker(),
    )

    method_logger = MethodLogger(
        method_name=method_name,
        qualitative_metrics={
            "Language": "Python",
            "Library": "T3F",
            "T3F backend": f"{backend_param}",
            "Tensor type": "Dense",
            "Data type": data_type_name,
            "Platform": "CPU, GPU",
            "Decomposition method": "TensorTrain",
        },
        method_args={
            "tens": tensor_param,
            "max_tt_rank": rank_param,
        },
        runner=method_runner,
    )

    reconstructed_tensor_t3f = method_runner.reconstructed_tensor

    method_logger.save_logs_to_file(is_test=False)

    reconstructed_frames = []

    for frame in reconstructed_tensor_t3f:
        reconstructed_frames.append(normalize_frames(frame))
    reconstructed_frames = np.array(reconstructed_frames)

    save_params_combinations = {
        "image": {"name": method_logger.name, "frames": reconstructed_frames},
        "video": {"name": method_logger.name, "frames": reconstructed_frames, "fps": original_fps, "frame_size": frame_size},
    }

    save_params = save_params_combinations[data_type_name]

    SaveFramesFactory.get_save_methods(frame_name=data_type_name).save_frames(**save_params)

Эксперимент набора параметров:   0%|          | 0/10 [00:00<?, ?it/s]I0000 00:00:1729181026.199732     884 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1729181026.199829     884 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1729181026.199885     884 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1729181026.205476     884 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1729181026.205522     884 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_

Изображение сохранено как /home/johndoe_19/git-projects/tensor-methods-comparison/.cache/output/T3F_TensorTrain_image.jpg
Пропущена итерация: логи уже существуют для T3F_TensorTrain_video


#### Clear cache and gc collect

In [25]:
torch.cuda.synchronize()
torch.cuda.empty_cache()

numba_device = cuda.get_current_device()
numba_device.reset()

variables_to_delete = [
    "logs",
    "tensor_rank_mapping",
    "input_tensors",
    "backend_param",
    "total_iterations",
    "data_type_name",
    "input_tensor",
    "rank_param",
    "tensor_param",
    "library_method_name",
    "method_name",
    "method_runner",
    "method_logger",
    "reconstructed_tensor_t3f",
    "reconstructed_frames",
    "frame",
    "save_params_combinations",
    "save_params",
]

for var in variables_to_delete:
    with contextlib.suppress(KeyError):
        del globals()[var]

gc.collect()

504

## TeNPy (not implemented)

### Params

In [26]:
# tensor_param = video_frames.copy().astype(np.float32)
#
# # Размерность физического индекса
# d = tensor_param.shape[-1]  # в вашем случае это 3
#
# # Создаем объект LegCharge
# leg = LegCharge.from_trivial(d)
#
# # Создаем объекты Site для каждого физического индекса, кроме последнего
# sites = [Site(leg) for _ in range(tensor_param.ndim - 1)]
#
# rank_param = [1, 500, 302, 500, 1]

### Implementation

In [27]:
# mps = MPS.from_full(sites=sites, psi=tensor_param, normalize=False)
#
# reconstructed_tensor = mps.to_full_tensor()

In [28]:
# logs = load_logs_from_file(log_file_path)
#
# method_name = f"TeNPy_TensorTrain"
#
# if logs:
#     existing_log = next(
#         (log for log in logs if log['method_name'] == method_name),
#         None
#     )
#     if existing_log:
#         error_message = f"Пропущена итерация: логи уже существуют для {method_name}"
#         raise error_message
#
# gc.collect()
# torch.cuda.empty_cache()
# torch.cuda.synchronize()
#
# method_logs = MethodLogger(
#     method_name=method_name,
#     method_input_tensor=tensor_param,
#     qualitative_metrics={
#         "Language": "Python",
#         "Library": "TeNPy",
#         "Tensor type": "Dense",
#         "Platform": "CPU",
#         "Decomposition method": "TensorTrain",
#     },
#     method_args={
#         "sites": tensor_param,
#         "psi": rank_param,
#         "normalize": False,
#     },
#     func=tenpy.networks.mps.MPS.from_full
# )
#
# method_logs_list.append(method_logs)
#
# tt_factors = method_logs.method_result
#
# reconstruct_frames_from_tenpy_tt_factors = tt_factors.to_full_tensor()

In [29]:
# method_logs.quantitative_metrics['compression_ratio'] = (100.0 * get_tensors_size(*tt_factors) / get_tensors_size(tensor_param))

# method_logs.quantitative_metrics['frobenius_error'] = (
#         100.0 * np.linalg.norm(reconstruct_frames_from_tenpy_tt_factors - tensor_param) / tl.norm(
#         tensor_param))
#
# save_logs_to_file(method_logs=method_logs, is_test=True)
#
# reconstructed_frames = []
#
# for i in range(len(reconstruct_frames_from_tenpy_tt_factors)):
#     reconstructed_frames.append(normalize_frame_tensorly_tensortrain(reconstruct_frames_from_tenpy_tt_factors[i]))
#
# save_frames_as_video(name=method_logs.name, frames=reconstructed_frames, fps=original_fps, frame_size=frame_size)